In [ ]:
import numpy as np
np.set_printoptions(threshold = np.nan, linewidth = 115)
import pickle
import keras
from keras import backend as K
from keras import models, optimizers, layers, regularizers, metrics, losses
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU, ReLU, ThresholdedReLU
from keras.layers.core import Dense, Dropout, SpatialDropout2D, Activation
from keras.layers.convolutional import Conv2D, SeparableConv2D
from keras.layers.pooling import MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import model_from_json, Sequential

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config = config)


IMG_SIZE = 256

# Save Comparison model
def save_model(model_name, hist_str, model_str):

    pickle.dump(model_name.history, open('Training Histories/'+ hist_str + '.p', 'wb'))
    
    print("Saved " + hist_str + " to Training Histories folder")
    
    # serialize model to JSON
    model_name = model.to_json()
    with open("CNN Models/" + model_str + ".json", "w") as json_file:
        json_file.write(model_name)

    # serialize weights to HDF5
    model.save_weights("CNN Models/" + model_str + ".h5")
    print("Saved " + model_str + " and weights to CNN Models folder")

# Load with pickle instead of processing images again
training_img_1 = pickle.load(open('training_img_1.p', 'rb'))
training_img_2 = pickle.load(open('training_img_2.p', 'rb'))
training_img = np.append(training_img_1, training_img_2, axis = 0)

test_img = pickle.load(open('pickle_test_img.p', 'rb'))

training_labels = pickle.load(open('training_labels.p', 'rb'))
test_labels = pickle.load(open('test_labels.p', 'rb'))    

# Metric Analysis
def true_accuracy(y_test, pred):
    
    pred = pred.astype(dtype = 'int32')
    
    ft = pred == y_test
    
    acc = []
    
    counter = 0
    while counter < len(ft):
        if sum(ft[counter]) < 15:
            acc.append(0)
            counter += 1
        else:
            acc.append(1)
            counter += 1
            
    # Accuracy       
    Acc = (sum(acc)/len(acc))
    
    print('\t Complete Label Accuracy: ', (Acc * 100), '%')
    
    print('Sum of Fully Correct Predictions: ', sum(acc))
    print('\t\t    Total Labels: ', len(acc))
    
    if Acc == 0:
        message = 'Feels Devastating (ﾉಥ益ಥ）ﾉ ┻━┻'
        
    elif Acc > 0 and Acc < 50:
        message = 'Feels Awful (੭ ˃̣̣̥ ㅂ˂̣̣̥)੭ु'
        
    elif Acc >= 50 and Acc < 60:
        message = 'Feels Bad (⌯˃̶᷄ ﹏ ˂̶᷄⌯)'
        
    elif Acc >= 60 and Acc < 70:
        message = 'Feels Meh... ┬─┬ノ(ಠ_ಠノ)'
    
    elif Acc >= 70 and Acc < 80:
        message = 'Feels Ok ʕ ·㉨·ʔ'
    
    elif Acc >= 80 and Acc < 90:
        message = 'Feels Better (^._.^)ﾉ'
        
    elif Acc >= 90 and Acc < 95:
        message = 'Feels Hopeful ( •́ ⍨ •̀)'
        
    elif Acc >= 95 and Acc < 98:
        message = 'Feels Good ヽ|･ω･|ゞ'
        
    elif Acc >= 98:
        message = 'Feels Great! ᕙ( * •̀ ᗜ •́ * )ᕗ'
        
    print('\n', message)
    
    return Acc

In [7]:
# v2: Added reg's to all layers
# v3: Changed No. of Outputs in Conv and Dense layers in Block 5 and Output
# v4: Added Batch Normalization to all layers
# v5: Using softmax final activation
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(SpatialDropout2D(0.5))
model.add(layers.BatchNormalization())

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Block 5
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.375))
model.add(layers.BatchNormalization())

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(15, activation='softmax'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), 
              loss = tf_accuracy, 
              metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 10, validation_split = (1 / 9), batch_size = 256, verbose = 1)

Predictions = model.predict(test_img)
Predictions = Predictions.astype(dtype = 'int32')

Accuracy = true_accuracy(test_labels, Predictions)

print('\t Complete Label Accuracy: ', (Accuracy * 100), '%')

history_str = 'vanilla_reg_VGG_v5_history'
model_str   = 'vanilla_reg_VGG_v5_20e'
    
save_model(model_obj, history_str, model_str)

TypeError: object of type 'bool' has no len()

In [6]:
# Metric Analysis
def tf_accuracy(y_test, pred):
    
    int_pred = tf.cast(pred, tf.int32)
    
    ft = int_pred == y_test
    
    acc = []
    
    counter = 0
    while counter < len(ft):
        if sum(ft[counter]) < 15:
            acc.append(0)
            counter += 1
        else:
            acc.append(1)
            counter += 1
            
    # Accuracy       
    Acc = (sum(acc)/len(acc))
    
    #print('\t Complete Label Accuracy: ', (Acc * 100), '%')
    
    #print('Sum of Fully Correct Predictions: ', sum(acc))
    #print('\t\t    Total Labels: ', len(acc))
    
    if Acc == 0:
        message = 'Feels Devastating (ﾉಥ益ಥ）ﾉ ┻━┻'
        
    elif Acc > 0 and Acc < 50:
        message = 'Feels Awful (੭ ˃̣̣̥ ㅂ˂̣̣̥)੭ु'
        
    elif Acc >= 50 and Acc < 60:
        message = 'Feels Bad (⌯˃̶᷄ ﹏ ˂̶᷄⌯)'
        
    elif Acc >= 60 and Acc < 70:
        message = 'Feels Meh... ┬─┬ノ(ಠ_ಠノ)'
    
    elif Acc >= 70 and Acc < 80:
        message = 'Feels Ok ʕ ·㉨·ʔ'
    
    elif Acc >= 80 and Acc < 90:
        message = 'Feels Better (^._.^)ﾉ'
        
    elif Acc >= 90 and Acc < 95:
        message = 'Feels Hopeful ( •́ ⍨ •̀)'
        
    elif Acc >= 95 and Acc < 98:
        message = 'Feels Good ヽ|･ω･|ゞ'
        
    elif Acc >= 98:
        message = 'Feels Great! ᕙ( * •̀ ᗜ •́ * )ᕗ'
        
    print('\n', message)
    
    return K.mean(Acc)

### VGG_Doppleganger

In [ ]:
# Directly taken from VGG16 with 2D Spatial Dropout, Dropout, and fewer Dense layers
'''
categorical_accuracy
'''
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(Activation('relu'))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.summary()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), 
              loss = 'binary_crossentropy', 
              metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, 
                      epochs = 30, validation_split = (1/9), 
                      batch_size = 128, verbose = 1)

history_str = 'VGG_Doppleganger_history'
model_str   = 'VGG_Doppleganger_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Removed Conv2D layer in Block 3 and moved SpDrop layer to Block 1
'''
categorical_accuracy
'''
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 20, validation_split = 0.1, batch_size = 128, verbose = 1)

history_str = 'VGG_Doppleganger_v3_history'
model_str   = 'vanilla_reg_VGG_v4_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Removed Conv2D layer in Block 3 and moved SpDrop layer to Block 1
# v3: Using Categorical Accuracy as metrics
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 20, validation_split = 0.1, batch_size = 128)

history_str = 'VGG_Doppleganger_v3_history'
model_str   = 'VGG_Doppleganger_v3_30e'
    
save_model(model_obj, history_str, model_str)

### vanilla_VGG_Dropouts

In [ ]:
# Based on VGG with half reg'd layers and Dropout layers
'''
categorical_accuracy
'''
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_VGG_Dropouts_history'
model_str   = 'vanilla_VGG_Dropouts_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_VGG_Dropouts_v2_history'
model_str   = 'vanilla_VGG_Dropouts_v2_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
# v3: Added reg's to all layers
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_VGG_Dropouts_v3_history'
model_str   = 'vanilla_VGG_Dropouts_v3_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
# v3: Added reg's to all layers
# v4: Changed Regularizer Weight Penalty (0.001 -> 0.005)
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_VGG_Dropouts_v4_history'
model_str   = 'vanilla_VGG_Dropouts_v4_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
# v3: Added reg's to all layers
# v4: Changed Regularizer Weight Penalty (0.001 -> 0.005)
# v5: Added Dropout to all layers previously without, changed percentage
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.25, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.25, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.375))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.375))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.375))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_VGG_Dropouts_v5_history'
model_str   = 'vanilla_VGG_Dropouts_v5_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
# v3: Added reg's to all layers
# v4: Changed Regularizer Weight Penalty (0.001 -> 0.005)
# v5: Added Dropout to all layers previously without, changed percentage
# v6: Added Batch Normalization layers to each Block
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.SpatialDropout2D(0.25, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.SpatialDropout2D(0.25, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.375))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.375))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.375))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_VGG_Dropouts_v6_history'
model_str   = 'vanilla_VGG_Dropouts_v6_30e'
    
save_model(model_obj, history_str, model_str)

### vanilla_reg_VGG

In [ ]:
# Modeled after VGGNet with half reg'd layers
vanilla_reg_VGG = models.Sequential()

# Block 1
vanilla_reg_VGG.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
vanilla_reg_VGG.add(layers.Conv2D(16, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Block 2
vanilla_reg_VGG.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(32, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Block 3
vanilla_reg_VGG.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(64, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Block 4
vanilla_reg_VGG.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(128, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Block 5
vanilla_reg_VGG.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(256, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Output Block
vanilla_reg_VGG.add(layers.Flatten())
vanilla_reg_VGG.add(layers.Dense(64, activation='relu'))
vanilla_reg_VGG.add(layers.Dense(15, activation='sigmoid'))

vanilla_reg_VGG.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = vanilla_reg_VGG.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_reg_VGG_history'
model_str   = 'vanilla_reg_VGG_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added reg's to all layers
'''
categorical_accuracy
'''
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_reg_VGG_v2_history'
model_str   = 'vanilla_reg_VGG_v2_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added reg's to all layers
# v3: Changed No. of Outputs in Conv and Dense layers in Block 5 and Output
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_reg_VGG_v3_history'
model_str   = 'vanilla_reg_VGG_v3_30e'
    
save_model(model_obj, history_str, model_str)

In [2]:
# v2: Added reg's to all layers
# v3: Changed No. of Outputs in Conv and Dense layers in Block 5 and Output
# v4: Added Batch Normalization to all layers
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Block 5
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), 
              loss = 'binary_crossentropy', 
              metrics = ['categorical_accuracy'])

'''
model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

Predictions = model.predict(test_img)
Predictions = Predictions.astype(dtype = 'int32')

Accuracy = true_accuracy(test_labels, Predictions)

history_str = 'vanilla_reg_VGG_v4_history'
model_str   = 'vanilla_reg_VGG_v4_30e'
    
save_model(model_obj, history_str, model_str)
'''

Train on 89696 samples, validate on 11212 samples
Epoch 1/30
89696/89696 [==============================] - 231s 3ms/step - loss: 0.7774 - categorical_accuracy: 0.5222 - val_loss: 0.6684 - val_categorical_accuracy: 0.4410
Epoch 2/30
89696/89696 [==============================] - 214s 2ms/step - loss: 0.5640 - categorical_accuracy: 0.5307 - val_loss: 0.5251 - val_categorical_accuracy: 0.3634
Epoch 3/30
89696/89696 [==============================] - 215s 2ms/step - loss: 0.4439 - categorical_accuracy: 0.5344 - val_loss: 0.4228 - val_categorical_accuracy: 0.5078
Epoch 4/30
89696/89696 [==============================] - 213s 2ms/step - loss: 0.3741 - categorical_accuracy: 0.5373 - val_loss: 0.3744 - val_categorical_accuracy: 0.4585
Epoch 5/30
89696/89696 [==============================] - 213s 2ms/step - loss: 0.3309 - categorical_accuracy: 0.5421 - val_loss: 0.3405 - val_categorical_accuracy: 0.4790
Epoch 6/30
89696/89696 [==============================] - 214s 2ms/step - loss: 0.3026 - c